In [60]:
from Bio import Entrez, SeqIO

Entrez.email = "cacultice@ucdavis.edu"

# Dictionary of species and their GenBank numbers
species_data = {
    "German Neanderthal": "AF011222",
    "Russian Neanderthal": "AF254446",
    "European Human": "X90314",
    "Mountain Gorilla Rwanda": "AF089820",
    "Chimp Troglodytes": "AF176766",
    "Puti Orangutan": "AF451972",
    "Jari Orangutan": "AF451964",
    "Western Lowland Gorilla": "AY079510",
    "Eastern Lowland Gorilla": "AF050738",
    "Chimp Schweinfurthii": "AF176722",
    "Chimp Vellerosus": "AF315498",
    "Chimp Verus": "AF176731"
}

# Fetch FASTA sequences
def fetch_fasta(accession):
    handle = Entrez.efetch(db = "nucleotide", id = accession, rettype = "fasta", retmode = "text")
    record = SeqIO.read(handle, "fasta")
    handle.close()
    return str(record.seq)  # Return as string

# Store in dictionary
real_dna_sequences = {species: fetch_fasta(acc) for species, acc in species_data.items()}

### Task 1

In [61]:
def EditDistance(X, Y):
    m = len(X)
    n = len(Y)
    c = [[0] * (n + 1) for _ in range(m + 1)] # Create matrix c with size (m+1) x (n+1), initialized to zero
    
    for i in range(m + 1): # Initialize the base case
        c[i][0] = i  # Cost of deleting all characters from X to match empty Y
    for j in range(n + 1):
        c[0][j] = j  # Cost of inserting all characters of Y to match empty X
    
    # Compute Edit Distance
    for i in range(1, m + 1): # loop through the length of X
        for j in range(1, n + 1): # loop through the length of y
            if X[i-1] == Y[j-1]: 
                cost = 0  # No cost if characters are the same
            else:
                cost = 1 # cost for substitution  # Cost of 1 for a substitution
            
            c[i][j] = min(c[i-1][j-1] + cost, # Substitution
                          c[i-1][j] + 1,  # cost for deletion
                          c[i][j-1] + 1) #cost for insertion
    
    return c[m][n]  # Edit distance is bottom right cell as it acts as the total

### Task 2

In [70]:
dist_matrix = [[0] * (len(real_dna_sequences)) for _ in range(len(real_dna_sequences))]

In [71]:
print(dist_matrix)

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


In [75]:
species_names = list(real_dna_sequences.keys())
species_seqs = list(real_dna_sequences.values())

In [76]:
for i in range(len(species_names)):
    for j in range(len(species_names)):
        dist_matrix[i][j] = EditDistance(species_seqs[i], species_seqs[j])

In [77]:
dist_matrix

[[0, 46, 48, 149, 106, 134, 135, 146, 151, 103, 156, 96],
 [46, 0, 73, 163, 80, 115, 110, 167, 165, 77, 126, 72],
 [48, 73, 0, 141, 88, 135, 132, 138, 140, 88, 166, 91],
 [149, 163, 141, 0, 158, 179, 177, 60, 17, 158, 142, 153],
 [106, 80, 88, 158, 0, 122, 115, 172, 160, 21, 115, 41],
 [134, 115, 135, 179, 122, 0, 11, 176, 181, 118, 166, 119],
 [135, 110, 132, 177, 115, 11, 0, 178, 180, 111, 167, 112],
 [146, 167, 138, 60, 172, 176, 178, 0, 60, 167, 143, 166],
 [151, 165, 140, 17, 160, 181, 180, 60, 0, 157, 143, 156],
 [103, 77, 88, 158, 21, 118, 111, 167, 157, 0, 107, 35],
 [156, 126, 166, 142, 115, 166, 167, 143, 143, 107, 0, 107],
 [96, 72, 91, 153, 41, 119, 112, 166, 156, 35, 107, 0]]

### Task 3 4, and 5

In [99]:
import numpy as np
import pandas as pd

def upgma(dist_matrix, species_list):
    clusters = {i: {species_list[i]} for i in range(len(species_list))}  # Store clusters as sets
    heights = {i: 0 for i in range(len(species_list))}  # Track heights
    T = {}  # Tree structure
    active_clusters = list(range(len(species_list)))  # Track active cluster indices

    print("Initial Distance Matrix:")
    df = pd.DataFrame(dist_matrix, index = species_names, columns = species_names)
    display(df)
    
    # Loop as long as there are multiple clusters
    while len(active_clusters) > 1:
        # Find the closest pair of clusters
        min_distance = float('inf')
        node1, node2 = -1, -1  # Initialize nodes as impossible values
        for i in range(len(dist_matrix)):  # Loop through dist_matrix to find the closest nodes
            for j in range(i):
                if dist_matrix[i][j] < min_distance:
                    min_distance = dist_matrix[i][j]
                    node1, node2 = i, j

        real_node1 = active_clusters[node1]
        real_node2 = active_clusters[node2]

        # Merge the clusters
        merged_cluster = clusters[real_node1] | clusters[real_node2]
        merge_cluster_index = max(clusters.keys(), default = -1) + 1  # Ensure unique index
        clusters[merge_cluster_index] = merged_cluster
        heights[merge_cluster_index] = min_distance / 2

        # Store merged clusters in the tree
        T[merge_cluster_index] = (real_node1, real_node2, min_distance / 2)

        print(f"\nMerging Clusters: {clusters[real_node1]} and {clusters[real_node2]} → {clusters[merge_cluster_index]} (Height: {min_distance / 2})")

        # Update Distance Matrix
        new_row = []  # Get new row for distance_matrix
        for k in range(len(dist_matrix)):
            if k != node1 and k != node2:  # Skip merged clusters
                new_dist = (dist_matrix[node1][k] + dist_matrix[node2][k]) / 2  # Applying formula
                new_row.append(new_dist)

        # Remove old clusters from distance matrix
        dist_matrix = np.delete(dist_matrix, [node1, node2], axis=0)
        dist_matrix = np.delete(dist_matrix, [node1, node2], axis=1)

        # Add the new cluster's distance values
        dist_matrix = np.vstack((dist_matrix, new_row))
        new_col = np.array(new_row + [0])[:, None]
        dist_matrix = np.hstack((dist_matrix, new_col))

        print(f"Updated Distance Matrix:\n{dist_matrix}")  # Print updated matrix

        # Update active cluster list
        new_active_clusters = []  # Create a new list for active clusters
        for i, c in enumerate(active_clusters):
            if i not in [node1, node2]:  # Skip merged clusters
                new_active_clusters.append(c)

        # Add merged cluster to the active list
        new_active_clusters.append(merge_cluster_index)
        active_clusters = new_active_clusters

        # Remove merged clusters
        del clusters[real_node1]
        del clusters[real_node2]
        del heights[real_node1]
        del heights[real_node2]

    print("\nFinal List of Clusters:")
    print_clusters(clusters)
    
    print("\nFinal Tree:")
    print_tree(T)
    
    return T

def print_tree(T):
    for parent, (child1, child2, height) in T.items():
        print(f"Cluster {parent} → ({child1}, {child2}) [Height: {height}]")

def print_clusters(clusters):
    print(f"{{ {', '.join(map(str, clusters.values()))} }}")


In [100]:
# Run algorithm on species DNA
upgma(dist_matrix, species_names)

Initial Distance Matrix:


,German Neanderthal,Russian Neanderthal,European Human,Mountain Gorilla Rwanda,Chimp Troglodytes,Puti Orangutan,Jari Orangutan,Western Lowland Gorilla,Eastern Lowland Gorilla,Chimp Schweinfurthii,Chimp Vellerosus,Chimp Verus
German Neanderthal,0,46,48,149,106,134,135,146,151,103,156,96
Russian Neanderthal,46,0,73,163,80,115,110,167,165,77,126,72
European Human,48,73,0,141,88,135,132,138,140,88,166,91
Mountain Gorilla Rwanda,149,163,141,0,158,179,177,60,17,158,142,153
Chimp Troglodytes,106,80,88,158,0,122,115,172,160,21,115,41
Puti Orangutan,134,115,135,179,122,0,11,176,181,118,166,119
Jari Orangutan,135,110,132,177,115,11,0,178,180,111,167,112
Western Lowland Gorilla,146,167,138,60,172,176,178,0,60,167,143,166
Eastern Lowland Gorilla,151,165,140,17,160,181,180,60,0,157,143,156
Chimp Schweinfurthii,103,77,88,158,21,118,111,167,157,0,107,35



Merging Clusters: {'Jari Orangutan'} and {'Puti Orangutan'} → {'Jari Orangutan', 'Puti Orangutan'} (Height: 5.5)
Updated Distance Matrix:
[[  0.   46.   48.  149.  106.  146.  151.  103.  156.   96.  134.5]
 [ 46.    0.   73.  163.   80.  167.  165.   77.  126.   72.  112.5]
 [ 48.   73.    0.  141.   88.  138.  140.   88.  166.   91.  133.5]
 [149.  163.  141.    0.  158.   60.   17.  158.  142.  153.  178. ]
 [106.   80.   88.  158.    0.  172.  160.   21.  115.   41.  118.5]
 [146.  167.  138.   60.  172.    0.   60.  167.  143.  166.  177. ]
 [151.  165.  140.   17.  160.   60.    0.  157.  143.  156.  180.5]
 [103.   77.   88.  158.   21.  167.  157.    0.  107.   35.  114.5]
 [156.  126.  166.  142.  115.  143.  143.  107.    0.  107.  166.5]
 [ 96.   72.   91.  153.   41.  166.  156.   35.  107.    0.  115.5]
 [134.5 112.5 133.5 178.  118.5 177.  180.5 114.5 166.5 115.5   0. ]]

Merging Clusters: {'Eastern Lowland Gorilla'} and {'Mountain Gorilla Rwanda'} → {'Eastern Lowland Go

{12: (6, 5, 5.5),
 13: (8, 3, 8.5),
 14: (9, 4, 10.5),
 15: (14, 11, 19.0),
 16: (1, 0, 23.0),
 17: (13, 7, 30.0),
 18: (16, 2, 30.25),
 19: (18, 15, 44.3125),
 20: (19, 12, 61.125),
 21: (17, 10, 71.375),
 22: (21, 20, 78.86328125)}